# Enhancements to RAG Pipelines (Optional)

While a basic RAG pipeline retrieves and feeds relevant documents to a language model, several enhancements can improve accuracy, reliability, and efficiency:

1. **Reranking**  
   - After retrieving candidate documents, a reranker (often a cross-encoder model) can reorder results based on semantic relevance.  
   - This ensures the most useful passages are prioritized before being passed to the language model.

2. **Prompt Engineering**  
   - Carefully designing prompts can guide the model to generate more accurate and structured answers.  
   - Examples include: asking for step-by-step reasoning, enforcing answer formats, or limiting the scope of responses.

3. **Handling Longer Context**  
   - Many queries require large amounts of background knowledge. Techniques such as chunking, summarization, and hierarchical retrieval allow the model to handle long documents.  
   - Recent models also support extended context windows, making it easier to process more text without losing coherence.

These enhancements make RAG systems more robust and adaptable to real-world Q&A scenarios.


In [ ]:
## reranking example

# imports and setup
import sys
import os
import json
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
import warnings
from langchain.schema import Document
warnings.filterwarnings('ignore')

project_root = os.path.abspath(os.path.join("..", ".."))
sys.path.append(project_root)

MODEL_NAME = "intfloat/e5-base-v2"
vec_store_path = os.path.join(project_root, "data", "vector_store", "faiss_index")
# load the embedding model  
embeddings = HuggingFaceBgeEmbeddings(model_name=MODEL_NAME, encode_kwargs={"normalize_embeddings": True})

vectorstore = FAISS.load_local(vec_store_path, embeddings, allow_dangerous_deserialization=True)

# dense retrieval
def dense_search(query, k=10):
    results = vectorstore.similarity_search_with_score(query, k=k)
    return results

# Example search
query = "Give me cake recipe with chocolate and frosting"
results_dense = dense_search(query)
for doc, score in results_dense:
    print(f"Score: {score:.4f}\nContent: {doc.page_content}\n")

In [ ]:
from sentence_transformers import CrossEncoder
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

docs_only = [doc for doc, _ in results_dense]
pairs = [(query, doc.page_content) for doc in docs_only]
scores = reranker.predict(pairs)
ranked = sorted(zip(docs_only, scores), key=lambda x: x[1], reverse=True)

print("\nReranked Results:\n")
for doc, score in ranked:
    print(f"Score: {score:.4f}\nContent: {doc.page_content}\n")

The ms-marco-MiniLM-L-6-v2 model is a cross-encoder specialized for ranking passages in search tasks. Instead of just embedding text, it directly compares a query with a passage and produces a relevance score. This makes it particularly effective for sifting through large sets of documents to highlight the passages most likely to answer a question. Built on the compact and efficient MiniLM framework and fine-tuned on the MS MARCO dataset, the model achieves a strong trade-off between speed and accuracy, making it well-suited for real-world information retrieval systems.

#### Note:
For your use case, you might want to include **reranking**. Even if the initial retrieval step brings back relevant documents, they may not always be in the best order. A reranker model (like a cross-encoder) can take the query and each retrieved passage together, score their relevance more precisely, and reorder the results. This helps ensure that the most useful passages are placed at the top before being passed to the language model for answering.



### Prompt Engineering

1. **Ground the Response in Context**  
   - Clearly instruct the model to use only the retrieved documents when answering.  
   - Example: *"Answer the question using only the context below. If the answer is not found, say 'I don’t know'."*

2. **Be Explicit About the Task**  
   - State exactly what you expect: summary, step-by-step reasoning, bullet points, or a direct answer.  
   - Avoid vague instructions like *"Tell me about this."*

3. **Limit the Scope**  
   - Prevent hallucinations by constraining the model’s role.  
   - Example: *"You are a research assistant. Only provide answers supported by the retrieved context."*

4. **Handle Long or Multiple Contexts**  
   - Use structured prompts (e.g., separating chunks with delimiters like `---`).  
   - This helps the model distinguish between different sources.

5. **Ask for Citations or Evidence**  
   - Encourage the model to point back to the passages it used.  
   - Example: *"After your answer, cite the context snippet you used."*

6. **Iterate and Test**  
   - Small changes in wording can lead to big differences in output.  
   - Always experiment with variations of the prompt to find what works best for your use case.



# Handling Larger Context

With many modern models offering generous context window sizes, you may not immediately face issues when working with short queries or smaller knowledge bases. However, challenges arise in **chat-based scenarios**, where the system needs to remember and build on past interactions. Over time, the accumulated history can exceed the model’s context limit.  

To manage this, techniques such as **summarizing earlier turns**, **retrieving only the most relevant parts of the conversation**, or **using memory modules** can help. These approaches ensure the model stays coherent and context-aware without being overloaded by the full chat history.
